# SQLite Data Pipeline
This is a simple seto of queries against a SQLite database using classic MS Northwind schema.  This represents a simple "baseline" of common types of queries.

Information on the North Wind Schema: [North Wind ER Diagram](/notebooks/docs/NorthwindERDiagram.png)

### Helper functions for SQLite
Simple functions to execute and return a formatted dataset.

In [3]:
import sqlite3
from pandas import *

WORKING_DIR = %env PWD
SQLITE_DB_FILE_NAME = '{0}{1}'.format(WORKING_DIR, '/data/Northwind.sl3')
    
def execute(sql):
    """ Simple code to execute SQL & return a Pandas DatTable."""   
    try:
        conn = sqlite3.connect(SQLITE_DB_FILE_NAME)
        conn.text_factory = str
        
        return pandas.read_sql_query(sql, conn)

    except sqlite3.Error, e:
        return 'Error {0}:'.format(e.args[0])
    
    finally:
        if conn:
            conn.close()


### Order Subtotals
For each order, calculate a subtotal for each Order (identified by OrderID). This is a simple query using GROUP BY to aggregate data for each order.

In [4]:
execute("""select OrderID, 
    sum(UnitPrice * Quantity * (1 - Discount)) as Subtotal
from "order details"
group by OrderID
order by OrderID""")

OrderID   Subtotal
0      10248   440.0000
1      10249  1863.4000
2      10250  1552.6000
3      10251   654.0600
4      10252  3597.9000
5      10253  1444.8000
6      10254   556.6200
7      10255  2490.5000
8      10256   517.8000
9      10257  1119.9000
10     10258  1614.8800
11     10259   100.8000
12     10260  1504.6500
13     10261   448.0000
14     10262   584.0000
15     10263  1873.8000
16     10264   695.6250
17     10265  1176.0000
18     10266   346.5600
19     10267  3536.6000
20     10268  1101.2000
21     10269   642.2000
22     10270  1376.0000
23     10271    48.0000
24     10272  1456.0000
25     10273  2037.2800
26     10274   538.6000
27     10275   291.8400
28     10276   420.0000
29     10277  1200.8000
..       ...        ...
800    11048   525.0000
801    11049   273.6000
802    11050   810.0000
803    11051    36.0000
804    11052  1332.0000
805    11053  3055.0000
806    11054   305.0000
807    11055  1727.5000
808    11056  3740.0000
809    11057    45.0000
810    11058   858.0000
811    11059  1838.0000
812    11060   266.0000
813    11061   510.0000
814    11062   406.4000
815    11063  1342.9500
816    11064  4330.4000
817    11065   189.4200
818    11066   928.7500
819    11067    86.8500
820    11068  2027.0800
821    11069   360.0000
822    11070  1629.9750
823    11071   484.5000
824    11072  5218.0000
825    11073   300.0000
826    11074   232.0850
827    11075   498.1000
828    11076   792.7500
829    11077  1255.7205

[830 rows x 2 columns]

### Sales by Year
This query shows how to get the year part from Shipped_Date column. A subtotal is calculated by a sub-query for each order. The sub-query forms a table and then joined with the Orders table.

In [5]:
execute("""select distinct date(a.ShippedDate) as ShippedDate, 
    a.OrderID, 
    b.Subtotal, 
    strftime('%Y',a.ShippedDate) as Year
from Orders a 
inner join
(
    -- Get subtotal for each order
    select distinct OrderID, 
        sum(UnitPrice * Quantity * (1 - Discount)) as Subtotal
    from "Order Details"
    group by OrderID    
) b on a.OrderID = b.OrderID
where a.ShippedDate is not null
    and a.ShippedDate between date('1996-12-24') and date('1997-09-30')
order by a.ShippedDate""")

Empty DataFrame
Columns: [ShippedDate, OrderID, Subtotal, Year]
Index: []

### Alphabetical List of Products
This is another simple query. No aggregation is used for summarizing data.

In [6]:
execute("""select distinct b.*,
    a.CategoryName 
from Categories a 
inner join Products b on a.CategoryID = b.CategoryID
where b.Discontinued = 0
order by b.ProductName""")

ProductID                      ProductName  SupplierID  CategoryID  \
0           3                    Aniseed Syrup           1           2   
1          40                 Boston Crab Meat          19           8   
2          60                Camembert Pierrot          28           4   
3          18                 Carnarvon Tigers           7           8   
4           1                             Chai           1           1   
5           2                            Chang           1           1   
6          39                 Chartreuse verte          18           1   
7           4     Chef Anton's Cajun Seasoning           2           2   
8          48                        Chocolade          22           3   
9          38                    C�te de Blaye          18           1   
10         58           Escargots de Bourgogne          27           8   
11         52                         Filo Mix          24           5   
12         71                      Flotemysost          15           4   
13         33                          Geitost          15           4   
14         15                     Genen Shouyu           6           2   
15         56           Gnocchi di nonna Alice          26           5   
16         31                Gorgonzola Telino          14           4   
17          6     Grandma's Boysenberry Spread           3           2   
18         37                       Gravad lax          17           8   
19         69                 Gudbrandsdalsost          15           4   
20         44                     Gula Malacca          20           2   
21         26              Gumb�r Gummib�rchen          11           3   
22         22              Gustaf's Kn�ckebr�d           9           5   
23         10                            Ikura           4           8   
24         36                      Inlagd Sill          17           8   
25         43                      Ipoh Coffee          20           1   
26         41  Jack's New England Clam Chowder          19           8   
27         13                            Konbu           6           8   
28         76                     Lakkalik��ri          23           1   
29         67        Laughing Lumberjack Lager          16           1   
..        ...                              ...         ...         ...   
39         25          NuNuCa Nu�-Nougat-Creme          11           3   
40         77  Original Frankfurter gr�ne So�e          12           2   
41         70                    Outback Lager           7           1   
42         16                          Pavlova           7           3   
43         55                     P�t� chinois          25           6   
44         11                   Queso Cabrales           5           4   
45         12        Queso Manchego La Pastora           5           4   
46         59             Raclette Courdavault          28           4   
47         57                   Ravioli Angelo          26           5   
48         75             Rh�nbr�u Klosterbier          12           1   
49         45                      Rogede sild          21           8   
50         73                       R�d Kaviar          17           8   
51         34                    Sasquatch Ale          16           1   
52         27               Schoggi Schokolade          11           3   
53         68              Scottish Longbreads           8           3   
54         20           Sir Rodney's Marmalade           8           3   
55         21              Sir Rodney's Scones           8           3   
56         61                   Sirop d'�rable          29           2   
57         46                        Spegesild          21           8   
58         35                   Steeleye Stout          16           1   
59         62                   Tarte au sucre          29           3   
60         19       Teatime Chocolate Biscuits           8           3   
61      

### Order Details Extended
This query calculates sales price for each order after discount is applied.

In [7]:
execute("""select distinct y.OrderID, 
    y.ProductID, 
    x.ProductName, 
    y.UnitPrice, 
    y.Quantity, 
    y.Discount, 
    y.UnitPrice * y.Quantity * (1 - y.Discount) as ExtendedPrice
from Products x
inner join "Order Details" y on x.ProductID = y.ProductID
order by y.OrderID""")

OrderID  ProductID                       ProductName  UnitPrice  \
0       10248         11                    Queso Cabrales      14.00   
1       10248         42     Singaporean Hokkien Fried Mee       9.80   
2       10248         72            Mozzarella di Giovanni      34.80   
3       10249         14                              Tofu      18.60   
4       10249         51             Manjimup Dried Apples      42.40   
5       10250         41   Jack's New England Clam Chowder       7.70   
6       10250         51             Manjimup Dried Apples      42.40   
7       10250         65  Louisiana Fiery Hot Pepper Sauce      16.80   
8       10251         22               Gustaf's Kn�ckebr�d      16.80   
9       10251         57                    Ravioli Angelo      15.60   
10      10251         65  Louisiana Fiery Hot Pepper Sauce      16.80   
11      10252         20            Sir Rodney's Marmalade      64.80   
12      10252         33                           Geitost       2.00   
13      10252         60                 Camembert Pierrot      27.20   
14      10253         31                 Gorgonzola Telino      10.00   
15      10253         39                  Chartreuse verte      14.40   
16      10253         49                          Maxilaku      16.00   
17      10254         24                Guaran� Fant�stica       3.60   
18      10254         55                      P�t� chinois      19.20   
19      10254         74                     Longlife Tofu       8.00   
20      10255          2                             Chang      15.20   
21      10255         16                           Pavlova      13.90   
22      10255         36                       Inlagd Sill      15.20   
23      10255         59              Raclette Courdavault      44.00   
24      10256         53                     Perth Pasties      26.20   
25      10256         77   Original Frankfurter gr�ne So�e      10.40   
26      10257         27                Schoggi Schokolade      35.10   
27      10257         39                  Chartreuse verte      14.40   
28      10257         77   Original Frankfurter gr�ne So�e      10.40   
29      10258          2                             Chang      15.20   
...       ...        ...                               ...        ...   
2125    11075         46                         Spegesild      12.00   
2126    11075         76                      Lakkalik��ri      18.00   
2127    11076          6      Grandma's Boysenberry Spread      25.00   
2128    11076         14                              Tofu      23.25   
2129    11076         19        Teatime Chocolate Biscuits       9.20   
2130    11077          2                             Chang      19.00   
2131    11077          3                     Aniseed Syrup      10.00   
2132    11077          4      Chef Anton's Cajun Seasoning      22.00   
2133    11077          6      Grandma's Boysenberry Spread      25.00   
2134    11077          7   Uncle Bob's Organic Dried Pears      30.00   
2135    11077          8        Northwoods Cranberry Sauce      40.00   
2136    11077         10                             Ikura      31.00   
2137    11077         12         Queso Manchego La Pastora      38.00   
2138    11077         13                             Konbu       6.00   
2139    11077         14                              Tofu      23.25   
2140    11077         16                           Pavlova      17.45   
2141    11077         20            Sir Rodney's Marmalade      81.00   
2142    11077         23                          Tunnbr�d       9.00   
2143    11077         32                Mascarpone Fabioli      32.00   
2144    11077         39                  Chartreuse verte      18.00   
2145    11077         41   Jack's New England Clam Chowder       9.65   
2146    11077         46                         Spegesild      12.00   
2147    11077         52                          Filo Mix      

### Sales by Category
For each category, we get the list of products sold and the total sales amount. Note that, the inner query for table c is to get sales for each product on each order. It then joins with outer query on Product_ID. In the outer query, products are grouped for each category.

In [8]:
execute("""select distinct a.CategoryID, 
    a.CategoryName, 
    b.ProductName, 
    sum(c.ExtendedPrice) as ProductSales
from Categories a 
inner join Products b on a.CategoryID = b.CategoryID
inner join 
(
    select distinct y.OrderID, 
        y.ProductID, 
        x.ProductName, 
        y.UnitPrice, 
        y.Quantity, 
        y.Discount, 
        y.UnitPrice * y.Quantity * (1 - y.Discount) as ExtendedPrice
    from Products x
    inner join "Order Details" y on x.ProductID = y.ProductID
    order by y.OrderID
) c on c.ProductID = b.ProductID
inner join Orders d on d.OrderID = c.OrderID
where d.OrderDate between '1997-01-01' and '1997-12-31'
group by a.CategoryID, a.CategoryName, b.ProductName
order by a.CategoryName, b.ProductName, ProductSales""")

Empty DataFrame
Columns: [CategoryID, CategoryName, ProductName, ProductSales]
Index: []

### Ten Most Expensive Products
Retrieves data from an ordered sub-query table and then the keyword LIMIT is used outside the sub-query to restrict the number of rows returned.

In [9]:
execute("""select * from
(
    select distinct ProductName as Ten_Most_Expensive_Products, 
           UnitPrice
    from Products
    order by UnitPrice desc
) as a
limit 10""")

Ten_Most_Expensive_Products  UnitPrice
0               C�te de Blaye     263.50
1     Th�ringer Rostbratwurst     123.79
2             Mishi Kobe Niku      97.00
3      Sir Rodney's Marmalade      81.00
4            Carnarvon Tigers      62.50
5        Raclette Courdavault      55.00
6       Manjimup Dried Apples      53.00
7              Tarte au sucre      49.30
8                 Ipoh Coffee      46.00
9           R�ssle Sauerkraut      45.60

### Customers and Suppliers by City
This query shows how to use UNION to merge Customers and Suppliers into one result set by identifying them as having different relationships to Northwind Traders - Customers and Suppliers.

In [10]:
execute("""select City, CompanyName, ContactName, 'Customers' as Relationship
from Customers
union
select City, CompanyName, ContactName, 'Suppliers'
from Suppliers
order by City, CompanyName""")

City                             CompanyName  \
0            Aachen                Drachenblut Delikatessen   
1       Albuquerque              Rattlesnake Canyon Grocery   
2         Anchorage                  Old World Delicatessen   
3         Ann Arbor               Grandma Kelly's Homestead   
4            Annecy                            Gai p�turage   
5         Barcelona                  Galer�a del gastr�nomo   
6      Barquisimeto                       LILA-Supermercado   
7              Bend                       Bigfoot Breweries   
8           Bergamo            Magazzini Alimentari Riuniti   
9            Berlin                     Alfreds Futterkiste   
10           Berlin             Heli S��waren GmbH & Co. KG   
11             Bern                       Chop-suey Chinese   
12            Boise                      Save-a-lot Markets   
13           Boston             New England Seafood Cannery   
14      Brandenburg                         K�niglich Essen   
15        Bruxelles                            Maison Dewey   
16           Br�cke                          Folk och f� HB   
17     Buenos Aires              Cactus Comidas para llevar   
18     Buenos Aires                  Oc�ano Atl�ntico Ltda.   
19     Buenos Aires                           Rancho grande   
20            Butte                         The Cracker Box   
21         Campinas                     Gourmet Lanchonetes   
22          Caracas                    GROSELLA-Restaurante   
23        Charleroi                        Supr�mes d�lices   
24             Cork            Hungry Owl All-Night Grocers   
25            Cowes                          Island Trading   
26        Cunewalde                              QUICK-Stop   
27         Cuxhaven  Nord-Ost-Fisch Handelsgesellschaft mbH   
28            Elgin              Hungry Coyote Import Store   
29           Eugene                 Great Lakes Food Market   
..              ...                                     ...   
90   Rio de Janeiro                      Ricardo Adocicados   
91          Salerno                    Pasta Buttini s.r.l.   
92         Salzburg                        Piccolo und mehr   
93    San Crist�bal                        HILARION-Abastos   
94    San Francisco                       Let's Stop N Shop   
95         Sandvika                         Norske Meierier   
96        Sao Paulo                        Com�rcio Mineiro   
97        Sao Paulo                      Familia Arquibaldo   
98        Sao Paulo                           Queen Cozinha   
99        Sao Paulo               Refrescos Americanas LTDA   
100       Sao Paulo                  Tradi��o Hipermercados   
101         Seattle                    White Clover Markets   
102         Sevilla                     Godos Cocina T�pica   
103       Singapore                            Leka Trading   
104         Stavern                           Sant� Gourmet   
105   Ste-Hyacinthe                        For�ts d'�rables   
106       Stockholm                       Svensk Sj�f�da AB   
107      Strasbourg                Blondesddsl p�re et fils   
108       Stuttgart                       Die Wandernde Kuh   
109          Sydney                             G'day, Mate   
110           Tokyo                           Tokyo Traders   
111          Torino                          Franchi S.p.A.   
112        Toulouse                        La maison d'Asie   
113       Tsawassen                   Bottom-Dollar Markets   
114       Vancouver           Laughing Bacchus Wine Cellars   
115      Versailles                    La corne d'abondance   
116     Walla Walla                    Lazy K Kountry Store   
117        Warszawa                          Wolski  Zajazd   
118         Zaandam                     Zaanse Snoepfabriek   
119           �rhus                            Vaffeljernet   

                 ContactName Relationship  
0               Sven Ottlieb    Customers  
1               Paul